# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

In [1]:
import os
from dotenv import main

import openai

In [2]:
# loading openai api key
_ = main.load_dotenv(main.find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
model_name = "gpt-4o"

### ConversationBufferMemory

In [4]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [5]:
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [6]:
llm = ChatOpenAI(model=model_name)

In [7]:
chain = RunnableWithMessageHistory(llm, get_session_history)
config = {"configurable" : {"session_id" : "1"}}

In [8]:
chain.invoke("Hi, I'm Manideep Bangaru",
             config=config)

AIMessage(content='Hello, Manideep Bangaru! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 15, 'total_tokens': 30, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_057232b607', 'finish_reason': 'stop', 'logprobs': None}, id='run-c11d64bb-80c3-4c27-8e06-1966a45bee43-0', usage_metadata={'input_tokens': 15, 'output_tokens': 15, 'total_tokens': 30})

#### Memory objects can also be incorporated into the get_session_history callable:

In [9]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationTokenBufferMemory, ConversationSummaryMemory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI


In [10]:
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]

    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=3,
        return_messages=True,
    ) # replace ConversationBufferWindowMemory with any of the other types of memory we might need
    assert len(memory.memory_variables) == 1
    key = memory.memory_variables[0]
    messages = memory.load_memory_variables({})[key]
    store[session_id] = InMemoryChatMessageHistory(messages=messages)
    return store[session_id]

In [11]:
llm = ChatOpenAI(model=model_name)

config = {"configurable": {"session_id": "1"}}

In [12]:
chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Manideep Bangaru",
    config=config,
)  # session_id determines thread

AIMessage(content='Hello, Manideep! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 14, 'total_tokens': 27, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c17d3befe7', 'finish_reason': 'stop', 'logprobs': None}, id='run-45d9f72d-9557-4e90-9b7b-66205e1cfd45-0', usage_metadata={'input_tokens': 14, 'output_tokens': 13, 'total_tokens': 27})